# Feature Engineering

#### Import the required libraries

In [9]:
# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_columns', 100)

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline 

# Seaborn for visualization
import seaborn as sns

#### Import the cleaned dataset

In [39]:
# Load cleaned dataset from the previous lecture
df = pd.read_csv('cleaned_df.csv')
df.head(2)

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type
0,295850,2013,234,81,1,1,584,2013,0,0.0,Condo
1,216500,2006,169,51,1,1,612,1965,0,1.0,Condo


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          1883 non-null   int64  
 1   year_sold      1883 non-null   int64  
 2   property_tax   1883 non-null   int64  
 3   insurance      1883 non-null   int64  
 4   beds           1883 non-null   int64  
 5   baths          1883 non-null   int64  
 6   sqft           1883 non-null   int64  
 7   year_built     1883 non-null   int64  
 8   lot_size       1883 non-null   int64  
 9   basement       1883 non-null   float64
 10  property_type  1883 non-null   object 
dtypes: float64(1), int64(9), object(1)
memory usage: 161.9+ KB


## I. Domain Knowledge

#### A. Popular Properties

2 bedroom and 2 bathroom properties are especially popular for investors. Let's create an indicator variable just for properties with 2 beds and 2 baths.

In [3]:
# Build your code step by step
#((df.beds == 2) & (df.baths == 2))
# ((df.beds == 2) & (df.baths == 2)).astype(int)

In [41]:
# Create indicator variable for properties with 2 beds and 2 baths
df['popular'] = ((df.beds == 2) & (df.baths == 2)).astype(int)

In [42]:
# Check how many propoerties have 2 baths and 2 beds 
df.popular.value_counts()

0    1705
1     178
Name: popular, dtype: int64

#### B. Housing Market Recession

We are modeling housing prices in the United States, it's important to consider the housing market recession around 2008. According to data from Zillow, the lowest housing prices were from 2010 to end of 2013.

<br>
Create an indicator feature **recession**

Here's how:
* Your first condition `year_sold >= 2010`
* Your second condition `year_sold <= 2013`
* Combine the two conditions with an `&` operator
* Convert the resulting data to `int` type.

In [43]:
# Create a new variable recession
df['recession']=((df.year_sold >= 2010) & (df.year_sold <= 2013)).astype(int)

In [44]:
# Check how many propoerties were sold during recession period 
df.recession.value_counts()

0    1387
1     496
Name: recession, dtype: int64

In [45]:
fd = df[['year_sold','recession']]
fd.head(8)

,year_sold,recession
0,2013,1
1,2006,0
2,2012,1
3,2005,0
4,2002,0
5,2004,0
6,2011,1
7,2005,0


## II. Interaction Features

In the first step, you engineered features from domain knowledge. interaction features can be products, sums, or differences between two features.

#### A. Property Age

We have the features `year_sold` and the `year_built`. let's create a new feature `property_age`

In [54]:
df['property_age'] = df.year_sold - df.year_built

In [52]:
type(df.property_age)

pandas.core.series.Series

Do a quick sanity check on that feature. Run `df.describe()` and check the stats for the feature `property_age`

In [55]:
# Do you see any error?
df.describe()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,popular,recession,property_age
count,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1.883000e+03,1883.000000,1883.000000,1883.000000,1883.000000
mean,422839.807754,2007.111524,466.777483,140.454063,3.420605,2.579926,2329.398832,1982.963887,1.339262e+04,0.879979,0.094530,0.263409,24.147637
std,151462.593276,5.196898,231.656645,72.929765,1.068554,0.945576,1336.991858,20.295945,4.494930e+04,0.325073,0.292642,0.440600,21.167604
min,200000.000000,1993.000000,88.000000,30.000000,1.000000,1.000000,500.000000,1880.000000,0.000000e+00,0.000000,0.000000,0.000000,-8.000000
25%,300000.000000,2004.000000,320.000000,94.000000,3.000000,2.000000,1345.000000,1970.000000,1.542000e+03,1.000000,0.000000,0.000000,6.000000
50%,392000.000000,2007.000000,426.000000,125.000000,3.000000,3.000000,1907.000000,1986.000000,6.098000e+03,1.000000,0.000000,0.000000,20.000000
75%,525000.000000,2011.000000,569.000000,169.000000,4.000000,3.000000,3005.000000,2000.000000,1.176100e+04,1.000000,0.000000,1.000000,38.000000
max,800000.000000,2016.000000,4508.000000,1374.000000,5.000000,6.000000,8450.000000,2015.000000,1.220551e+06,1.000000,1.000000,1.000000,114.000000


In [57]:
# Check number of observations with 'property_age' < 0
(df.property_age < 0).sum()

19

On second thought, this could be an error or that some homeowners buy houses before the construction company builts them. But for the purpose of this project we will remove these observations.

We'll do a quick ad-hoc data cleaning and remove these observations from our dataset.

#### Remove observations where `property_age` is less than 0.
* Keep only observations where `property_age` is 0 and above.

In [14]:
# Print df shape before
print(df.shape)

# Remove rows where property_age is less than 0
df = df[df.property_age >= 0]

# Print number of rows in remaining dataframe
print(df.shape)

(1882, 15)
(1863, 15)


In [58]:
df.corr()

C:\Users\Samuel\AppData\Local\Temp\ipykernel_24484\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,popular,recession,property_age
price,1.000000,0.108782,0.535148,0.532947,0.384046,0.389375,0.469573,0.033138,0.130558,0.109479,-0.147166,-0.023845,-0.005066
year_sold,0.108782,1.000000,-0.257703,-0.254706,-0.174081,-0.132110,-0.236190,-0.043301,-0.048207,-0.070075,0.068880,0.510217,0.287029
property_tax,0.535148,-0.257703,1.000000,0.984724,0.547643,0.525776,0.660264,-0.046504,0.165800,0.184327,-0.207048,-0.114896,-0.018680
insurance,0.532947,-0.254706,0.984724,1.000000,0.485776,0.475430,0.594049,-0.109521,0.146579,0.138771,-0.173698,-0.114483,0.042478
beds,0.384046,-0.174081,0.547643,0.485776,1.000000,0.644572,0.691039,-0.011844,0.164399,0.414633,-0.429675,-0.067287,-0.031383
baths,0.389375,-0.132110,0.525776,0.475430,0.644572,1.000000,0.682060,0.206141,0.132976,0.344108,-0.198216,-0.053111,-0.230087
sqft,0.469573,-0.236190,0.660264,0.594049,0.691039,0.682060,1.000000,0.254589,0.246016,0.270308,-0.268666,-0.073216,-0.302093
year_built,0.033138,-0.043301,-0.046504,-0.109521,-0.011844,0.206141,0.254589,1.000000,0.000068,0.008846,0.100235,0.031012,-0.969452
lot_size,0.130558,-0.048207,0.165800,0.146579,0.164399,0.132976,0.246016,0.000068,1.000000,0.011123,-0.078069,-0.005772,-0.011900
basement,0.109479,-0.070075,0.184327,0.138771,0.414633,0.344108,0.270308,0.008846,0.011123,1.000000,-0.199046,-0.046260,-0.025686


### III. Drop Redundant Features

Beacuse we created a new feature `property_age` using features `year_built` and `year_sold`. We can drop those two features.

**Remove features 'year_built' and 'year_sold'** 
* Use Pandas's `.drop()` function.
* Remember to set `axis=1` because you are dropping columns.
* Remember to do 'inplace=True'.


In [36]:
# Drop 'year_built' and 'year_sold' from the dataset
df.drop(['year_built', 'year_sold'], axis=1, inplace=True)
df.head()

,Unnamed: 0,price,property_tax,insurance,beds,baths,sqft,lot_size,basement,property_type,popular,recession
0,0,295850,234,81,1,1,584,0,0.0,Condo,0,1
1,1,216500,169,51,1,1,612,0,1.0,Condo,0,0
2,2,279900,216,74,1,1,615,0,0.0,Condo,0,1
3,3,379900,265,92,1,1,618,33541,0.0,Condo,0,0
4,4,340000,88,30,1,1,634,0,0.0,Condo,0,0


### Save the final dataset

We will save this dataset and train our model on it.

In [59]:
# Save the data as 'final.csv'
df.to_csv('final.csv', index=None)